In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv")
faults.head()

C:\Users\nitin\AppData\Local\Temp\ipykernel_30252\3681478151.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [3]:
#removed rows from certina long and latititue . Lets go with 80 kms using haverson.
# Locations to drop around
drop_locations = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the distance in kilometers between two points on Earth 
    specified by their longitudes and latitudes in decimal degrees.
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 
    return c * r

# Mark rows to drop
radius_km = 80 #50 miles radius
def is_near_any(row, locations, radius):
    for lat, lon in locations:
        if haversine(row['Latitude'], row['Longitude'], lat, lon) <= radius:
            return True
    return False

# Apply and filter
faults['drop'] = faults.apply(lambda row: is_near_any(row, drop_locations, radius_km), axis=1)
faults_cleaned = faults[~faults['drop']].drop(columns='drop')

In [4]:
faults_cleaned

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187329,1248453,123901925,2020-03-06 13:44:45.000,Low (Severity Medium) Engine Coolant Level,NaN,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,False,93,NaN,1886,105351219,39.013472,-77.037685,2020-03-06 13:44:41.000
1187331,1248455,123905139,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [5]:
faults_selected = faults_cleaned[['RecordID','EventTimeStamp','eventDescription','spn','fmi','active','activeTransitionCount','EquipmentID','Latitude','Longitude','LocationTimeStamp']]
faults_selected.head()

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp
0,1,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638,2015-02-21 11:39:37.000


In [6]:
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")

diagnostics.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [7]:
diag_pivoted = diagnostics.pivot(index='FaultId', columns='Name', values='Value').reset_index()

In [8]:
diag_pivoted

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100,14.5,True,64.6226,423937.9,185,51,37.12,211.4937,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0,14.355,True,66.48672,465925.4,186.8,62,41.18,212.8438,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
1187333,1248457,1.6,14.4275,False,67.72946,28606.65625,181.4,0,27.26,221.7312,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [9]:
#Lets merge faults and diagnosis
faults_diag_merged = pd.merge(faults_selected, diag_pivoted, how='left', left_on='RecordID', right_on='FaultId')
faults_diag_merged

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737592,1248453,2020-03-06 13:44:45.000,Low (Severity Medium) Engine Coolant Level,111,18,False,93,1886,39.013472,-77.037685,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
737593,1248455,2020-03-06 14:04:23.000,Condition Exists Engine Protection Torque Derate,1569,31,True,5,1994,34.390740,-79.461805,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
737594,1248456,2020-03-06 14:13:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1850,34.430370,-84.920509,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
737595,1248457,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,111,18,True,8,2377,35.030925,-85.321527,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [10]:
#convert time stamp from object to date
faults_diag_merged['EventTimeStamp'] = pd.to_datetime(faults_diag_merged['EventTimeStamp'],errors='coerce')
faults_diag_merged['LocationTimeStamp'] = pd.to_datetime(faults_diag_merged['LocationTimeStamp'],errors='coerce')
faults_diag_merged

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,2015-02-21 11:34:34,NaN,629,12,True,127,1439,38.857638,-84.626851,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-02-21 11:39:41,NaN,4364,17,False,2,1674,38.416481,-89.442638,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737592,1248453,2020-03-06 13:44:45,Low (Severity Medium) Engine Coolant Level,111,18,False,93,1886,39.013472,-77.037685,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
737593,1248455,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,1569,31,True,5,1994,34.390740,-79.461805,...,32,True,98.6,18431,False,NaN,65.01096,NaN,73.2,7.83
737594,1248456,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1850,34.430370,-84.920509,...,NaN,True,91.4,17407,NaN,NaN,66.5741,NaN,100,6.96
737595,1248457,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,111,18,True,8,2377,35.030925,-85.321527,...,NaN,True,100.4,1023,False,NaN,11.84489,14.1,100,1.74


In [11]:
# faults_targetted = faults_diag_merged[faults_diag_merged['EquipmentID']
# .isin(list(faults_diag_merged[ (faults_diag_merged['spn'] == 5246) & 
#            (faults_diag_merged['active'] == True) &
#            (faults_diag_merged['EventTimeStamp'] <= '2018-12-31')]['EquipmentID']
#            .unique()))]
# Lets take data before 12/31/2018 as training data and build on top of that
faults_targetted = faults_diag_merged[(faults_diag_merged['EventTimeStamp'] > '2018-12-31')]

# Sort just to be safe
temp_sort_df = faults_targetted.sort_values(by=['EquipmentID', 'EventTimeStamp'])

# List to collect all event frames
all_events = []

# Loop through each TruckID
for truck_id, truck_df in temp_sort_df.groupby('EquipmentID'):
    truck_df = truck_df.sort_values('EventTimeStamp')
    
    # Find 5246 occurrences
    hits_5246 = truck_df[truck_df['spn'] == 5246]['EventTimeStamp'].reset_index(drop=True)
    
    event_counter = 1  # Reset counter for each truck

    for i, current_hit_time in enumerate(hits_5246):
        # Window: last 24 hours including current hit
        window_start = current_hit_time - pd.Timedelta(hours=14)

        # Find if any previous 5246 in that window
        previous_5246 = hits_5246[hits_5246 < current_hit_time]
        previous_5246_in_window = previous_5246[previous_5246 >= window_start]

        if not previous_5246_in_window.empty:
            # If previous 5246 found within window, start from that
            adjusted_start_time = previous_5246_in_window.max()
        else:
            # Otherwise normal 24-hour window
            adjusted_start_time = window_start
        
        mask = (truck_df['EventTimeStamp'] >= adjusted_start_time) & (truck_df['EventTimeStamp'] <= current_hit_time) & (truck_df['active'] == True)
        window_df = truck_df[mask].copy()

        # Drop if only 5246 exists in window
        if window_df['spn'].nunique() == 1 and (5246 in window_df['spn'].unique()):
            # Only 5246 present → Skip this window
            continue

        # Sort by Time just to be safe
        window_df = window_df.sort_values('EventTimeStamp')

        # De-duplication: Check (spn, Latitude, Longitude) against ALL previous
        seen = set()
        deduped_rows = []

        for idx, row in window_df.iterrows():
            key = (row['spn'], row['Latitude'], row['Longitude'])
            if key not in seen:
                seen.add(key)
                deduped_rows.append(row)

        # Create cleaned DataFrame
        deduped_window_df = pd.DataFrame(deduped_rows)
        
        # Label the Event
        deduped_window_df['Event'] = f"{truck_id}_5246_Event#{event_counter}"
        
        all_events.append(deduped_window_df)
        
        event_counter += 1

# Combine all events together
final_df = pd.concat(all_events, ignore_index=True)

# Drop exact duplicates across events
final_df = final_df.drop_duplicates(subset=['EventTimeStamp', 'spn', 'Latitude', 'Longitude'])

# Get the 5246 times for each Event
event_5246_times = final_df[final_df['spn'] == 5246].groupby('Event')['EventTimeStamp'].min()

# Merge that info back into final_df
final_df['Event_5246_Time'] = final_df['Event'].map(event_5246_times)

# Now calculate Hours Before 5246
final_df['Hours_Before_5246'] = (final_df['Event_5246_Time'] - final_df['EventTimeStamp']).dt.total_seconds() / 3600


# print("\nFinal Data with Event Labels:")
# #print(final_df)

# # # Now Pivot: Create summary table
# pivot_wider = final_df.pivot_table(
#     index='Event',
#     columns='spn',
#     values='EquipmentID',  # any non-null field works
#     aggfunc='count',
#     fill_value=0
# )

# Group by TruckID, Event, and Code
# summary_df = final_df.groupby(['EquipmentID', 'Event', 'spn']).size().reset_index(name='Count')
# summary_df

In [12]:
final_df

,Event,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,...,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,Event_5246_Time,Hours_Before_5246
0,1698_5246_Event#1,1105397.0,2019-01-13 10:15:19,Not Reporting Data Front Operator Wiper Switch,2863.0,7.0,True,61.0,1698.0,37.963240,...,87.8,255,False,NaN,49.21454,NaN,100,17.11,2019-01-13 21:21:37,11.105000
1,1698_5246_Event#1,1105470.0,2019-01-13 14:01:50,Not Reporting Data Front Operator Wiper Switch,2863.0,7.0,True,64.0,1698.0,36.136111,...,87.8,255,False,NaN,66.40904,NaN,100,12.76,2019-01-13 21:21:37,7.329722
2,1698_5246_Event#1,1105477.0,2019-01-13 15:16:16,Incorrect Data J1939 Network #1 Primary Vehicl...,639.0,2.0,True,127.0,1698.0,36.139212,...,102.2,1279,False,NaN,4.349598,NaN,100,1.16,2019-01-13 21:21:37,6.089167
3,1698_5246_Event#1,1105551.0,2019-01-13 21:21:37,NaN,5246.0,0.0,True,1.0,1698.0,34.698842,...,111.2,22527,True,NaN,0,NaN,100,0,2019-01-13 21:21:37,0.000000
4,1698_5246_Event#2,1105650.0,2019-01-14 07:14:09,Low (Severity Medium) Engine Coolant Level,111.0,18.0,True,2.0,1698.0,34.698888,...,77,22527,True,NaN,0,NaN,100,0.87,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,2089_5246_Event#1,1211173.0,2019-11-01 17:28:12,NaN,5246.0,16.0,True,1.0,2089.0,32.191018,...,98.6,18431,False,NaN,66.67119,14.15,100,4.35,2019-11-01 17:28:12,0.000000
143,2211_5246_Event#1,1221939.0,2019-12-06 06:16:16,NaN,5394.0,5.0,True,2.0,2211.0,34.404305,...,93.2,17407,True,NaN,0,NaN,100,1.16,2019-12-06 08:28:31,2.204167
144,2211_5246_Event#1,1221979.0,2019-12-06 08:28:31,NaN,5246.0,16.0,True,1.0,2211.0,33.470416,...,102.2,18431,False,NaN,67.37023,NaN,100,15.95,2019-12-06 08:28:31,0.000000
145,2211_5246_Event#3,1223046.0,2019-12-10 13:12:57,NaN,5394.0,5.0,True,3.0,2211.0,31.561527,...,89.6,17407,False,NaN,2.776752,NaN,100,0.58,2019-12-10 14:16:24,1.057500


### Grab the selected columns for target data sets

In [13]:
derate_codes = pd.read_csv("derate_codes.csv")

In [14]:
faults_target_data = faults_targetted[['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active','AcceleratorPedal','BarometricPressure','CruiseControlSetSpeed',
'EngineCoolantTemperature','EngineLoad','EngineOilPressure','EngineOilTemperature','EngineRpm','EngineTimeLtd','FuelLevel','FuelLtd','FuelTemperature',
'IgnStatus','IntakeManifoldTemperature','LampStatus','ServiceDistance','Speed','SwitchedBatteryVoltage','Throttle','TurboBoostPressure']]

In [15]:

(derate_codes['spn'].values)

array([ 639,  111,  639,   94, 1761, 1761, 3362, 3251, 3216, 3226, 3482,
        641,   96, 3226, 3361, 3610, 3610, 3556, 4340, 4342, 4765, 3242,
       3246,   91, 3226, 4344,   84,  171, 4094, 4096, 3361, 5394, 5394,
       4334, 4334, 4334, 4364, 3360, 5024, 3031, 1569, 3216, 3364, 3364,
       5848, 5742, 5743, 5745, 3031, 1761, 6802, 1761, 4339,  245,  251,
       4094], dtype=int64)

### Here we will take each row from target data set where event date is < 12/30/2018 and based on the derate_codes, create new column ind_derate = 1 OR 0

In [16]:
def is_derate_codes(row, derate_codes):
    if((row['spn'] in list(derate_codes['spn'].values)) &
       (row['fmi'] in list(derate_codes[derate_codes['spn'] == row['spn']]['fmi'])) &
       (row['active'] == True)
      ):
       return 1
    else:
        return 0

In [17]:
#pass the codes
faults_target_data['is_derate_code'] = faults_target_data.apply(lambda row: is_derate_codes(row, derate_codes), axis=1)

C:\Users\nitin\AppData\Local\Temp\ipykernel_30252\3759071785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faults_target_data['is_derate_code'] = faults_target_data.apply(lambda row: is_derate_codes(row, derate_codes), axis=1)


In [18]:
faults_target_data['is_derate_code'].value_counts()

is_derate_code
0    64662
1    12294
Name: count, dtype: int64

### Now loop through the data and grab all diagnostic data for records where ind_derate = 1 per equipment ID. We will use these as predictors. 

In [19]:
faults_target_data

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code
660470,1100306,1808,2018-12-31 00:22:32,111,18,True,97.6,14.2825,65.86535,188.6,...,NaN,True,78.8,2047,NaN,51.61264,NaN,100,19.43,0
660471,1100307,1808,2018-12-31 00:28:30,111,18,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0
660472,1100310,1962,2018-12-31 01:23:14,111,18,True,0,14.2825,64.6226,62.6,...,NaN,True,55.4,2047,NaN,0,NaN,100,0.29,0
660473,1100311,1962,2018-12-31 01:25:52,111,18,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0
660474,1100321,1808,2018-12-31 03:23:32,111,18,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737592,1248453,1886,2020-03-06 13:44:45,111,18,False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,0
737593,1248455,1994,2020-03-06 14:04:23,1569,31,True,100,14.5,64.6226,185,...,32,True,98.6,18431,NaN,65.01096,NaN,73.2,7.83,1
737594,1248456,1850,2020-03-06 14:13:38,3216,10,True,0,14.355,66.48672,186.8,...,NaN,True,91.4,17407,NaN,66.5741,NaN,100,6.96,1
737595,1248457,2377,2020-03-06 14:14:13,111,18,True,1.6,14.4275,67.72946,181.4,...,NaN,True,100.4,1023,NaN,11.84489,14.1,100,1.74,0


In [20]:
faults_target_data = faults_target_data.drop(['LampStatus','ServiceDistance'],axis = 1)

fault_test = faults_target_data.copy()

columns = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature',
       'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

for col in columns:

    fault_test[col] = pd.to_numeric(fault_test[col], errors='coerce')


    fault_test[col] = fault_test.groupby('EquipmentID')[col].transform(
        lambda x: x.fillna(x.mean())
    )


    fault_test[col] = fault_test[col].fillna(fault_test[col].mean())

In [22]:
fault_test = fault_test[fault_test['spn'] != '5246']
columns_to_use2 = ['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active','AcceleratorPedal','BarometricPressure','CruiseControlSetSpeed','EngineCoolantTemperature','EngineLoad','EngineOilPressure','EngineOilTemperature','EngineRpm','EngineTimeLtd','FuelLevel','FuelLtd','FuelTemperature','IgnStatus','IntakeManifoldTemperature','Speed','SwitchedBatteryVoltage','Throttle','TurboBoostPressure','is_derate_code']

try2 = fault_test[columns_to_use2]

columns = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature',
       'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

for col in columns:

    try2[col] = pd.to_numeric(try2[col], errors='coerce')


    try2[col] = try2.groupby('EquipmentID')[col].transform(
        lambda x: x.fillna(x.mean())
    )


    try2[col] = try2[col].fillna(try2[col].mean())

In [23]:
variables2 = ['RecordID','EquipmentID','EventTimeStamp','spn','fmi','active','AcceleratorPedal','BarometricPressure','CruiseControlSetSpeed','EngineCoolantTemperature','EngineLoad','EngineOilPressure','EngineOilTemperature','EngineRpm','EngineTimeLtd','FuelLevel','FuelLtd','FuelTemperature', 'IntakeManifoldTemperature','Speed','SwitchedBatteryVoltage','Throttle','TurboBoostPressure','is_derate_code']
X_test = try2[variables2]

In [25]:
import joblib
pipeline = joblib.load('bigG_model_randomForest.pkl')
# 0.25 is threshold

#Predict with  pipeline and threshold
y_proba = pipeline.predict_proba(X_test)[:, 1]
y_pred_final = (y_proba >= 0.25).astype(int)

# Map back to X_test
X_test_with_predictions = X_test.copy()
X_test_with_predictions['predicted_5246'] = y_pred_final
X_test_with_predictions['probability_5246'] = y_proba

# 3. See final result
X_test_with_predictions.head()

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelLtd,FuelTemperature,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code,predicted_5246,probability_5246
660470,1100306,1808,2018-12-31 00:22:32,111,18,True,97.600000,14.282500,65.865350,188.600000,...,61167.585694,33.344209,78.800000,51.612640,13.592109,100.000000,19.430000,0,0,0.05
660471,1100307,1808,2018-12-31 00:28:30,111,18,False,21.282759,14.076250,65.640370,186.055172,...,67307.183304,33.344209,92.982759,63.413340,13.592109,94.503448,11.385000,0,0,0.12
660472,1100310,1962,2018-12-31 01:23:14,111,18,True,0.000000,14.282500,64.622600,62.600000,...,38624.199465,33.344209,55.400000,0.000000,13.592109,100.000000,0.290000,0,0,0.06
660473,1100311,1962,2018-12-31 01:25:52,111,18,False,37.169231,14.076154,64.415477,159.938462,...,44779.279577,33.344209,83.276923,34.886158,13.592109,97.128205,7.093846,0,0,0.09
660474,1100321,1808,2018-12-31 03:23:32,111,18,False,21.282759,14.076250,65.640370,186.055172,...,67307.183304,33.344209,92.982759,63.413340,13.592109,94.503448,11.385000,0,0,0.12


In [26]:
X_test_with_predictions

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,active,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,EngineCoolantTemperature,...,FuelLtd,FuelTemperature,IntakeManifoldTemperature,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,is_derate_code,predicted_5246,probability_5246
660470,1100306,1808,2018-12-31 00:22:32,111,18,True,97.600000,14.282500,65.865350,188.600000,...,61167.585694,33.344209,78.800000,51.612640,13.592109,100.000000,19.430000,0,0,0.05
660471,1100307,1808,2018-12-31 00:28:30,111,18,False,21.282759,14.076250,65.640370,186.055172,...,67307.183304,33.344209,92.982759,63.413340,13.592109,94.503448,11.385000,0,0,0.12
660472,1100310,1962,2018-12-31 01:23:14,111,18,True,0.000000,14.282500,64.622600,62.600000,...,38624.199465,33.344209,55.400000,0.000000,13.592109,100.000000,0.290000,0,0,0.06
660473,1100311,1962,2018-12-31 01:25:52,111,18,False,37.169231,14.076154,64.415477,159.938462,...,44779.279577,33.344209,83.276923,34.886158,13.592109,97.128205,7.093846,0,0,0.09
660474,1100321,1808,2018-12-31 03:23:32,111,18,False,21.282759,14.076250,65.640370,186.055172,...,67307.183304,33.344209,92.982759,63.413340,13.592109,94.503448,11.385000,0,0,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737592,1248453,1886,2020-03-06 13:44:45,111,18,False,26.958621,14.083750,65.640368,178.172414,...,61570.839777,32.000000,94.348276,53.955174,13.592109,94.544828,8.475000,0,0,0.02
737593,1248455,1994,2020-03-06 14:04:23,1569,31,True,100.000000,14.500000,64.622600,185.000000,...,58979.184416,32.000000,98.600000,65.010960,13.592109,73.200000,7.830000,1,0,0.13
737594,1248456,1850,2020-03-06 14:13:38,3216,10,True,0.000000,14.355000,66.486720,186.800000,...,65080.105870,32.000000,91.400000,66.574100,13.592109,100.000000,6.960000,1,0,0.05
737595,1248457,2377,2020-03-06 14:14:13,111,18,True,1.600000,14.427500,67.729460,181.400000,...,4042.492826,33.344209,100.400000,11.844890,14.100000,100.000000,1.740000,0,0,0.02
